In [41]:
#Getting symbols

with open(r'data/symbols_ibovf.csv', 'r') as file:
    symbols = [line.strip() for line in file if line.strip()]



In [42]:
import api.metatrader_api as mt5_api

#setting up server 
mt5_server = mt5_api.MetaTrader_API(_login=58034070, _password='Ms$12345', _server='XPMT5-DEMO')

#historical data download
historical_data = mt5_server.historical_data_cluster(symbols, '2023-02-01', '2023-03-01', 'M10')

In [43]:
historical_data['BOVA11']

,time,open,high,low,close,tick_volume,spread,real_volume
date,,,,,,,,
2023-02-01 00:00:00,1675177200,109.51,109.62,109.44,109.59,1009,1,93100
2023-02-01 00:10:00,1675177800,109.60,109.63,109.55,109.59,548,1,10716
2023-02-01 00:20:00,1675178400,109.60,109.65,109.54,109.63,797,1,137725
2023-02-01 00:30:00,1675179000,109.63,109.65,109.57,109.64,727,1,27604
2023-02-01 00:40:00,1675179600,109.64,109.70,109.62,109.68,636,1,26136
...,...,...,...,...,...,...,...,...
2023-02-28 23:20:00,1677594000,102.17,102.18,102.07,102.14,1310,1,39231
2023-02-28 23:30:00,1677594600,102.14,102.20,102.06,102.17,1820,1,61160
2023-02-28 23:40:00,1677595200,102.15,102.58,102.06,102.44,3190,1,185402


In [44]:
from statsmodels.tsa.stattools import adfuller
from arch.unitroot import engle_granger
import numpy as np
import pandas as pd


def volatility(df):
    data = []
    for i in range(len(df)-1):
        data.append((df[i+1]-df[i])/df[i])

    return data

class coint:


    def __new__(self,df,tf='', threshold = 0.1, column = 'close', save=False, n_lags=10):

        symbol_list = list(df.keys())

        raw_start = str(df[symbol_list[0]].index[0])
        raw_end =  str(df[symbol_list[0]].index[-1])
        date_start = '{}'.format(raw_start.split(' ')[0])
        date_end = '{}'.format(str(raw_end).split(' ')[0])

        coint_list = {'pvalue':[],'x':[], 'y':[]}

        size = len(df.keys()) * len(df.keys())

        for i in df.keys():

            for j in df.keys():

                if(i != j):
                    try:
                        eg = engle_granger(df[i][column], df[j][column], lags=n_lags)
                    except:
                        continue

                    pvalue = eg.pvalue

                    if(pvalue <= threshold):

                        coint_list['pvalue'].append(float(pvalue))
                        coint_list['x'].append(i)
                        coint_list['y'].append(j)

        coint_df = pd.DataFrame(coint_list)

        return coint_df

In [45]:
coint(historical_data)

['B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BPAC11', 'BRML3', 'CIEL3', 'CYRE3', 'IGTA3', 'IRBR3', 'ITSA4', 'ITUB4', 'MULT3', 'SANB11', 'SULA11', 'BOVA11']


,pvalue,x,y
0,0.095616,BBSE3,BBDC3
1,0.035471,BBSE3,BBDC4
2,0.093235,BBSE3,CIEL3
3,0.092052,BBSE3,SANB11
4,0.017316,IRBR3,BOVA11
5,0.002485,BOVA11,IRBR3
